In [1]:
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None

In [2]:
state_conversion = {1:'Alabama',
2:'Alaska',
4:'Arizona',
5:'Arkansas',
6:'California',
8:'Colorado',
9:'Connecticut',
10:'Delaware',
11:'District of Columbia',
12:'Florida',
13:'Georgia',
15:'Hawaii',
16:'Idaho',
17:'Illinois',
18:'Indiana',
19:'Iowa',
20:'Kansas',
21:'Kentucky',
22:'Louisiana',
23:'Maine',
24:'Maryland',
25:'Massachusetts',
26:'Michigan',
27:'Minnesota',
28:'Mississippi',
29:'Missouri',
30:'Montana',
31:'Nebraska',
32:'Nevada',
33:'New Hampshire',
34:'New Jersey',
35:'New Mexico',
36:'New York',
37:'North Carolina',
38:'North Dakota',
39:'Ohio',
40:'Oklahoma',
41:'Oregon',
42:'Pennsylvania',
44:'Rhode Island',
45:'South Carolina',
46:'South Dakota',
47:'Tennessee',
48:'Texas',
49:'Utah',
50:'Vermont',
51:'Virginia',
53:'Washington',
54:'West Virginia',
55:'Wisconsin',
56:'Wyoming'}

In [3]:
data16 = pd.read_csv("/Users/tristanmoser/files/tristan/488/project/NHSA/weather/w2016.csv")
data15 = pd.read_csv("/Users/tristanmoser/files/tristan/488/project/NHSA/weather/w2015.csv")
data14 = pd.read_csv("/Users/tristanmoser/files/tristan/488/project/NHSA/weather/w2014.csv")
data13 = pd.read_csv("/Users/tristanmoser/files/tristan/488/project/NHSA/weather/w2013.csv")
data12 = pd.read_csv("/Users/tristanmoser/files/tristan/488/project/NHSA/weather/w2012.csv")
data11 = pd.read_csv("/Users/tristanmoser/files/tristan/488/project/NHSA/weather/w2011.csv")
data10 = pd.read_csv("/Users/tristanmoser/files/tristan/488/project/NHSA/weather/w2010.csv")

In [4]:
data16['accdate'] = (data16['accdate'] - 2016)/10000 +10000
data15['accdate'] = (data15['accdate'] - 2015)/10000 +10000
data14['accdate'] = (data14['accdate'] - 2014)/10000 +10000
data13['accdate'] = (data13['accdate'] - 2013)/10000 +10000
data12['accdate'] = (data12['accdate'] - 2012)/10000 +10000
data11['accdate'] = (data11['accdate'] - 2011)/10000 +10000
data10['accdate'] = (data10['accdate'] - 2010)/10000 +10000

In [5]:
convert = {1:'Clear', 2:'Rain', 3:'Sleet or Hail', 4:'Snow', 5:'Fog or Smoke', 6:'Severe Crosswinds', 
          7:'Blowing Sand or Dirt', 8:'Other', 10:'Cloudy', 11:'Blowing Snow', 12: 'Freezing Rain'}

In [6]:
data16 = data16.replace({"atmcond": convert})
data15 = data15.replace({"atmcond": convert})
data14 = data14.replace({"atmcond": convert})
data13 = data13.replace({"atmcond": convert})
data12 = data12.replace({"atmcond": convert})
data11 = data11.replace({"atmcond": convert})
data10 = data10.replace({"atmcond": convert})

In [7]:
def date_change(dat):
    dat['accdate'] = dat['accdate'].astype(float) -10000
    dat['week'] = 0
    dat['day'] = 0
    return dat

In [8]:
data16 = date_change(data16)
data15 = date_change(data15)
data14 = date_change(data14)
data13 = date_change(data13)
data12 = date_change(data12)
data11 = date_change(data11)
data10 = date_change(data10)

In [9]:
data16 = data16.dropna().sort_values(by='accdate')
data15 = data15.dropna().sort_values(by='accdate')
data14 = data14.dropna().sort_values(by='accdate')
data13 = data13.dropna().sort_values(by='accdate')
data12 = data12.dropna().sort_values(by='accdate')
data11 = data11.dropna().sort_values(by='accdate')
data10 = data10.dropna().sort_values(by='accdate')

In [10]:
dataframes = [data16,data15,data14,data13,data12,data11,data10]

In [11]:
for dataframe in dataframes:
    df = dataframe['accdate'].unique()
    df = pd.Series(df)
    dfs = pd.DataFrame(columns=[['date','day']])
    dfs['date'] = df
    dfs['day'] = df.index
    change = {dfs['date'][ii]: dfs['day'][ii] + 1 for ii in range(len(dfs))}
    for ii in range(1,len(dataframe)):
        dataframe['day'][ii] = change[dataframe['accdate'][ii]]

In [12]:
data16.head()

,statenum,casenum,atmcond,county,accdate,dayofweek,week,day
9409,42.0,26.0,Clear,3.0,101.0,6.0,0,1
6290,6.0,560.0,Clear,59.0,101.0,6.0,0,1
21530,32.0,13.0,Clear,31.0,101.0,6.0,0,1
6909,48.0,95.0,Rain,29.0,101.0,6.0,0,1
23467,17.0,7.0,Clear,7.0,101.0,6.0,0,1


In [13]:
dataframes = [data16,data15,data14,data13,data12,data11,data10]

In [14]:
for dataframe in dataframes:
    dataframe['geo'] = 0
    for ii in range(1,len(dataframe)):
        dataframe['geo'][ii] = dataframe['statenum'][ii]*1000 + dataframe['county'][ii]

In [15]:
data16.head()

,statenum,casenum,atmcond,county,accdate,dayofweek,week,day,geo
9409,42.0,26.0,Clear,3.0,101.0,6.0,0,1,42003
6290,6.0,560.0,Clear,59.0,101.0,6.0,0,1,6059
21530,32.0,13.0,Clear,31.0,101.0,6.0,0,1,32031
6909,48.0,95.0,Rain,29.0,101.0,6.0,0,1,48029
23467,17.0,7.0,Clear,7.0,101.0,6.0,0,1,17007


In [16]:
dataframes = [data16,data15,data14,data13,data12,data11,data10]

In [17]:
def week1_gen(dat,x):
    for ii in range(1,len(dat)):
        if dat['day'][ii] < x:
            dat['week'][ii] = 1
        elif ((dat['day'][ii] >= x) & (dat['day'][ii] < (x+7))):
            dat['week'][ii] = 2
        elif ((dat['day'][ii] >= (x+7)) & (dat['day'][ii] < (x+14))):
            dat['week'][ii] = 3
        elif ((dat['day'][ii] >= (x+14) & (dat['day'][ii] < (x+21)))):
            dat['week'][ii] = 4
        else:
            pass

def week2_gen(dat,x):
    for ii in range(1,len(dat)):
        if ((dat['day'][ii] >= (x+21)) & (dat['day'][ii] < (x+28))):
            dat['week'][ii] = 5
        elif ((dat['day'][ii] >= (x+28)) & (dat['day'][ii] < (x+35))):
            dat['week'][ii] = 6
        elif ((dat['day'][ii] >= (x+35)) & (dat['day'][ii] < (x+42))):
            dat['week'][ii] = 7
        elif ((dat['day'][ii] >= (x+42)) & (dat['day'][ii] < (x+49))):
            dat['week'][ii] = 8
        else:
            pass

def week3_gen(dat,x):
    for ii in range(1,len(dat)):
        if ((dat['day'][ii] >= (x+49)) & (dat['day'][ii] < (x+56))):
            dat['week'][ii] = 9
        elif ((dat['day'][ii] >= (x+56)) & (dat['day'][ii] < (x+63))):
            dat['week'][ii] = 10
        elif ((dat['day'][ii] >= (x+63)) & (dat['day'][ii] < (x+70))):
            dat['week'][ii] = 11
        elif ((dat['day'][ii] >= (x+70)) & (dat['day'][ii] < (x+77))):
            dat['week'][ii] = 12
        else:
            pass

def week4_gen(dat,x):
    for ii in range(1,len(dat)):
        if ((dat['day'][ii] >= (x+(11*7))) & (dat['day'][ii] < (x+(12*7)))):
            dat['week'][ii] = 13
        elif ((dat['day'][ii] >= (x+(12*7))) & (dat['day'][ii] < (x+(13*7)))):
            dat['week'][ii] = 14
        elif ((dat['day'][ii] >= (x+(13*7))) & (dat['day'][ii] < (x+(14*7)))):
            dat['week'][ii] = 15
        elif ((dat['day'][ii] >= (x+(14*7))) & (dat['day'][ii] < (x+(15*7)))):
            dat['week'][ii] = 16
        else:
            pass

def week5_gen(dat,x):
    for ii in range(1,len(dat)):
        if ((dat['day'][ii] >= (x+(15*7))) & (dat['day'][ii] < (x+(16*7)))):
            dat['week'][ii] = 17
        elif ((dat['day'][ii] >= (x+(16*7))) & (dat['day'][ii] < (x+(17*7)))):
            dat['week'][ii] = 18
        elif ((dat['day'][ii] >= (x+(17*7))) & (dat['day'][ii] < (x+(18*7)))):
            dat['week'][ii] = 19
        elif ((dat['day'][ii] >= (x+(18*7))) & (dat['day'][ii] < (x+(19*7)))):
            dat['week'][ii] = 20
        else:
            pass

def week6_gen(dat,x):
    for ii in range(1,len(dat)):
        if ((dat['day'][ii] >= (x+(19*7))) & (dat['day'][ii] < (x+(20*7)))):
            dat['week'][ii] = 21
        elif ((dat['day'][ii] >= (x+(20*7))) & (dat['day'][ii] < (x+(21*7)))):
            dat['week'][ii] = 22
        elif ((dat['day'][ii] >= (x+(21*7))) & (dat['day'][ii] < (x+(22*7)))):
            dat['week'][ii] = 23      
        elif ((dat['day'][ii] >= (x+(22*7))) & (dat['day'][ii] < (x+(23*7)))):
            dat['week'][ii] = 24
        else:
            pass

def week7_gen(dat,x):
    for ii in range(1,len(dat)):
        if ((dat['day'][ii] >= (x+(23*7))) & (dat['day'][ii] < (x+(24*7)))):
            dat['week'][ii] = 25
        elif ((dat['day'][ii] >= (x+(24*7))) & (dat['day'][ii] < (x+(25*7)))):
            dat['week'][ii] = 26
        elif ((dat['day'][ii] >= (x+(25*7))) & (dat['day'][ii] < (x+(26*7)))):
            dat['week'][ii] = 27
        elif ((dat['day'][ii] >= (x+(26*7))) & (dat['day'][ii] < (x+(27*7)))):
            dat['week'][ii] = 28
        else:
            pass

def week8_gen(dat,x):
    for ii in range(1,len(dat)):
        if ((dat['day'][ii] >= (x+(27*7))) & (dat['day'][ii] < (x+(28*7)))):
            dat['week'][ii] = 29
        elif ((dat['day'][ii] >= (x+(28*7))) & (dat['day'][ii] < (x+(29*7)))):
            dat['week'][ii] = 30
        elif ((dat['day'][ii] >= (x+(29*7))) & (dat['day'][ii] < (x+(30*7)))):
            dat['week'][ii] = 31
        elif ((dat['day'][ii] >= (x+(30*7))) & (dat['day'][ii] < (x+(31*7)))):
            dat['week'][ii] = 32
        else:
            pass

def week9_gen(dat,x):
    for ii in range(1,len(dat)):
        if ((dat['day'][ii] >= (x+(31*7))) & (dat['day'][ii] < (x+(32*7)))):
            dat['week'][ii] = 33
        elif ((dat['day'][ii] >= (x+(32*7))) & (dat['day'][ii] < (x+(33*7)))):
            dat['week'][ii] = 34
        elif ((dat['day'][ii] >= (x+(33*7))) & (dat['day'][ii] < (x+(34*7)))):
            dat['week'][ii] = 35
        elif ((dat['day'][ii] >= (x+(34*7))) & (dat['day'][ii] < (x+(35*7)))):
            dat['week'][ii] = 36
        else:
            pass

def week10_gen(dat,x):
    for ii in range(1,len(dat)):
        if ((dat['day'][ii] >= (x+(35*7))) & (dat['day'][ii] < (x+(36*7)))):
            dat['week'][ii] = 37
        elif ((dat['day'][ii] >= (x+(36*7))) & (dat['day'][ii] < (x+(37*7)))):
            dat['week'][ii] = 38
        elif ((dat['day'][ii] >= (x+(37*7))) & (dat['day'][ii] < (x+(38*7)))):
            dat['week'][ii] = 39
        elif ((dat['day'][ii] >= (x+(38*7))) & (dat['day'][ii] < (x+(39*7)))):
            dat['week'][ii] = 40
        else:
            pass

def week11_gen(dat,x):
    for ii in range(1,len(dat)):
        if ((dat['day'][ii] >= (x+(39*7))) & (dat['day'][ii] < (x+(40*7)))):
            dat['week'][ii] = 41
        elif ((dat['day'][ii] >= (x+(40*7))) & (dat['day'][ii] < (x+(41*7)))):
            dat['week'][ii] = 42
        elif ((dat['day'][ii] >= (x+(41*7))) & (dat['day'][ii] < (x+(42*7)))):
            dat['week'][ii] = 43
        elif ((dat['day'][ii] >= (x+(42*7))) & (dat['day'][ii] < (x+(43*7)))):
            dat['week'][ii] = 44
        else:
            pass
def week12_gen(dat,x):
    for ii in range(1,len(dat)):
        if ((dat['day'][ii] >= (x+(43*7))) & (dat['day'][ii] < (x+(44*7)))):
            dat['week'][ii] = 45
        elif ((dat['day'][ii] >= (x+(44*7))) & (dat['day'][ii] < (x+(45*7)))):
            dat['week'][ii] = 46
        elif ((dat['day'][ii] >= (x+(45*7))) & (dat['day'][ii] < (x+(46*7)))):
            dat['week'][ii] = 47
        elif ((dat['day'][ii] >= (x+(46*7))) & (dat['day'][ii] < (x+(47*7)))):
            dat['week'][ii] = 48
        else:
            pass
def week13_gen(dat,x):
    for ii in range(1,len(dat)):
        if ((dat['day'][ii] >= (x+(47*7))) & (dat['day'][ii] < (x+(48*7)))):
            dat['week'][ii] = 49
        elif ((dat['day'][ii] >= (x+(48*7))) & (dat['day'][ii] < (x+(49*7)))):
            dat['week'][ii] = 50
        elif ((dat['day'][ii] >= (x+(49*7))) & (dat['day'][ii] < (x+(50*7)))):
            dat['week'][ii] = 51
        elif ((dat['day'][ii] >= (x+(50*7))) & (dat['day'][ii] < (x+(51*7)))):
            dat['week'][ii] = 52
        else:
            pass
def week14_gen(dat,x):
    for ii in range(1,len(dat)):
        if ((dat['day'][ii] >= (x+(51*7)))):
            dat['week'][ii] = 53

In [18]:
weeks = [week1_gen,week2_gen,week3_gen,week4_gen,week5_gen,week6_gen,week7_gen,week8_gen,
         week9_gen,week10_gen,week11_gen,week12_gen,week13_gen,week14_gen]

In [19]:
for week in weeks:
        week(data16,3)
        week(data15,4)
        week(data14,5)
        week(data13,6)
        week(data12,8)
        week(data11,2)
        week(data10,3)

In [20]:
dataframes2 = [data16,data15,data14,data13,data12,data11,data10]

In [21]:
data16.tail()

,statenum,casenum,atmcond,county,accdate,dayofweek,week,day,geo
2139,55.0,492.0,Cloudy,53.0,1231.0,7.0,53,366,55053
19745,37.0,1252.0,Clear,49.0,1231.0,7.0,53,366,37049
4040,42.0,1090.0,Clear,89.0,1231.0,7.0,53,366,42089
25823,48.0,3429.0,Rain,287.0,1231.0,7.0,53,366,48287
16693,48.0,3456.0,Cloudy,109.0,1231.0,7.0,53,366,48109


In [22]:
weekstep = pd.concat(dataframes)

In [23]:
data16['year'] = 2016
data15['year'] = 2015
data14['year'] = 2014
data13['year'] = 2013
data12['year'] = 2012
data11['year'] = 2011
data10['year'] = 2010

In [24]:
dataframes3 = [data16,data15,data14,data13,data12,data11,data10]
weekstep = pd.concat(dataframes3)

In [25]:
weekstep.head()

,statenum,casenum,atmcond,county,accdate,dayofweek,week,day,geo,year
9409,42.0,26.0,Clear,3.0,101.0,6.0,1,1,42003,2016
6290,6.0,560.0,Clear,59.0,101.0,6.0,1,1,6059,2016
21530,32.0,13.0,Clear,31.0,101.0,6.0,1,1,32031,2016
6909,48.0,95.0,Rain,29.0,101.0,6.0,1,1,48029,2016
23467,17.0,7.0,Clear,7.0,101.0,6.0,1,1,17007,2016


In [26]:
data16['unit'] = data16['geo']*1000 + data16['week']
data15['unit'] = data15['geo']*1000 + data15['week']
data14['unit'] = data14['geo']*1000 + data14['week']
data13['unit'] = data13['geo']*1000 + data13['week']
data12['unit'] = data12['geo']*1000 + data12['week']
data11['unit'] = data11['geo']*1000 + data11['week']
data10['unit'] = data10['geo']*1000 + data10['week']

In [27]:
years = [2010,2011,2012,2013,2014,2015]

In [28]:
weekstep['unit'] = weekstep['geo']*1000 + weekstep['week']

In [29]:
def final(dat,year):
    weather = []
    crash = []
    state = []
    years2 = []
    weeks1 = []
    units1 = []
    for county in weekstep['geo'].unique():
        #for year in years:
        for week in range(1,54):
            data1 = dat.drop(dat[((dat['geo'] != county))].index)
            data2 = data1.drop(data1[(data1['year'] != year)].index)
            data3 = data2.drop(data2[(data2['week'] != week)].index)
            counts = len(data3)
            #counts = np.sum(data2['fatalities'])
            values = data3['atmcond'].value_counts().keys().tolist()
            stat = data3['statenum'].value_counts().keys().tolist()
            yea = year
            wee = week
            uni = county
            units1.append(uni)
            try:
                weather.append(values[0])
            except IndexError:
                weather.append('None')
            try:
                state.append(stat[0])
            except IndexError:
                state.append('None')
            weeks1.append(wee)
            years2.append(yea)
            crash.append(counts)
    final = pd.DataFrame(columns=[['unit','weather']])
    final['unit'] = units1
    final['weather'] = weather
    final['crashes'] = crash
    final['state'] = state
    final['year'] = years2
    final['week'] = weeks1
    #for ii in range(len(final)):
    #    final['state'][ii] = state_conversion[final['state'][ii]]
    return final

In [30]:
weekstep.head()

,statenum,casenum,atmcond,county,accdate,dayofweek,week,day,geo,year,unit
9409,42.0,26.0,Clear,3.0,101.0,6.0,1,1,42003,2016,42003001
6290,6.0,560.0,Clear,59.0,101.0,6.0,1,1,6059,2016,6059001
21530,32.0,13.0,Clear,31.0,101.0,6.0,1,1,32031,2016,32031001
6909,48.0,95.0,Rain,29.0,101.0,6.0,1,1,48029,2016,48029001
23467,17.0,7.0,Clear,7.0,101.0,6.0,1,1,17007,2016,17007001


In [31]:
weekstep.head()

,statenum,casenum,atmcond,county,accdate,dayofweek,week,day,geo,year,unit
9409,42.0,26.0,Clear,3.0,101.0,6.0,1,1,42003,2016,42003001
6290,6.0,560.0,Clear,59.0,101.0,6.0,1,1,6059,2016,6059001
21530,32.0,13.0,Clear,31.0,101.0,6.0,1,1,32031,2016,32031001
6909,48.0,95.0,Rain,29.0,101.0,6.0,1,1,48029,2016,48029001
23467,17.0,7.0,Clear,7.0,101.0,6.0,1,1,17007,2016,17007001


In [32]:
test15 = final(data15,2015)

In [33]:
#test14 = final(data14,2014)

In [34]:
#test13 = final(data13,2013)

In [35]:
test15.head()

,unit,weather,crashes,state,year,week
0,42003,Other,1,42,2015,1
1,42003,None,0,None,2015,2
2,42003,None,0,None,2015,3
3,42003,Clear,1,42,2015,4
4,42003,Clear,1,42,2015,5


In [36]:
def clean_state(dat):
    dat['conv'] = dat['unit']/1000
    dat['unit'] = dat['unit'].astype(str)
    for ii in range(len(dat)):
        if dat['conv'][ii] < 10:
            dat['state'][ii] = dat['unit'][ii][0:1]
        else:
            dat['state'][ii] = dat['unit'][ii][0:2]
    dat['state'] = dat['state'].astype(int)
    for ii in range(len(dat)):
        if dat['state'][ii] != 0:
            dat['state'][ii] = state_conversion[dat['state'][ii]]
        else:
            pass
    return dat

In [48]:
def clean_weather(dat):
    for ii in range(len(dat)):
        if dat['weather'][ii] == 'None':
            try:
                dat['weather'][ii] = dat['weather'][ii-1]
            except KeyError:
                dat['weather'][ii] = 'Clear'
        else:
            pass
    return dat

In [38]:
test15 = clean_state(test15)

In [39]:
test15.head()

,unit,weather,crashes,state,year,week,conv
0,42003,Other,1,Pennsylvania,2015,1,42.003
1,42003,None,0,Pennsylvania,2015,2,42.003
2,42003,None,0,Pennsylvania,2015,3,42.003
3,42003,Clear,1,Pennsylvania,2015,4,42.003
4,42003,Clear,1,Pennsylvania,2015,5,42.003


In [41]:
test15 = clean_weather(test15)

In [42]:
test15.head(15)

,unit,weather,crashes,state,year,week,conv
0,42003,Other,1,Pennsylvania,2015,1,42.003
1,42003,Other,0,Pennsylvania,2015,2,42.003
2,42003,Other,0,Pennsylvania,2015,3,42.003
3,42003,Clear,1,Pennsylvania,2015,4,42.003
4,42003,Clear,1,Pennsylvania,2015,5,42.003
5,42003,Clear,0,Pennsylvania,2015,6,42.003
6,42003,Clear,1,Pennsylvania,2015,7,42.003
7,42003,Snow,2,Pennsylvania,2015,8,42.003
8,42003,Snow,0,Pennsylvania,2015,9,42.003
9,42003,Snow,0,Pennsylvania,2015,10,42.003


In [44]:
test14 = final(data14,2014)
test14 = clean_state(test14)
test14 = clean_weather(test14)

In [45]:
test14.head(15)

,unit,weather,crashes,state,year,week,conv
0,42003,Clear,1,Pennsylvania,2014,1,42.003
1,42003,Snow,1,Pennsylvania,2014,2,42.003
2,42003,Other,1,Pennsylvania,2014,3,42.003
3,42003,Other,0,Pennsylvania,2014,4,42.003
4,42003,Snow,2,Pennsylvania,2014,5,42.003
5,42003,Clear,1,Pennsylvania,2014,6,42.003
6,42003,Clear,1,Pennsylvania,2014,7,42.003
7,42003,Clear,1,Pennsylvania,2014,8,42.003
8,42003,Clear,1,Pennsylvania,2014,9,42.003
9,42003,Snow,2,Pennsylvania,2014,10,42.003


In [49]:
test13 = final(data13,2013)
test13 = clean_state(test13)
test13 = clean_weather(test13)

In [55]:
test13.head(15)

,unit,weather,crashes,state,year,week,conv
0,42003,Clear,0,Pennsylvania,2013,1,42.003
1,42003,Clear,0,Pennsylvania,2013,2,42.003
2,42003,Clear,2,Pennsylvania,2013,3,42.003
3,42003,Clear,0,Pennsylvania,2013,4,42.003
4,42003,Clear,3,Pennsylvania,2013,5,42.003
5,42003,Clear,1,Pennsylvania,2013,6,42.003
6,42003,Clear,3,Pennsylvania,2013,7,42.003
7,42003,Clear,0,Pennsylvania,2013,8,42.003
8,42003,Clear,1,Pennsylvania,2013,9,42.003
9,42003,Clear,0,Pennsylvania,2013,10,42.003


In [56]:
test13['weather'].unique()

array(['Clear', 'Rain', 'Snow', 'Cloudy', 'Severe Crosswinds',
       'Fog or Smoke', 'Freezing Rain', 'Blowing Snow', 'Other',
       'Sleet or Hail', 'Blowing Sand or Dirt'], dtype=object)

In [57]:
test12 = final(data12,2012)
test12 = clean_state(test12)
test12 = clean_weather(test12)

In [59]:
test12.head(15)

,unit,weather,crashes,state,year,week,conv
0,42003,Clear,0,Pennsylvania,2012,1,42.003
1,42003,Rain,2,Pennsylvania,2012,2,42.003
2,42003,Rain,0,Pennsylvania,2012,3,42.003
3,42003,Clear,1,Pennsylvania,2012,4,42.003
4,42003,Clear,0,Pennsylvania,2012,5,42.003
5,42003,Snow,2,Pennsylvania,2012,6,42.003
6,42003,Clear,1,Pennsylvania,2012,7,42.003
7,42003,Rain,1,Pennsylvania,2012,8,42.003
8,42003,Clear,2,Pennsylvania,2012,9,42.003
9,42003,Rain,3,Pennsylvania,2012,10,42.003


In [60]:
test11 = final(data11,2011)
test11 = clean_state(test11)
test11 = clean_weather(test11)

In [69]:
test11.head(15)

,unit,weather,crashes,state,year,week,conv
0,42003,Clear,0,Pennsylvania,2011,1,42.003
1,42003,Clear,0,Pennsylvania,2011,2,42.003
2,42003,Clear,2,Pennsylvania,2011,3,42.003
3,42003,Clear,2,Pennsylvania,2011,4,42.003
4,42003,Clear,0,Pennsylvania,2011,5,42.003
5,42003,Clear,0,Pennsylvania,2011,6,42.003
6,42003,Clear,2,Pennsylvania,2011,7,42.003
7,42003,Clear,0,Pennsylvania,2011,8,42.003
8,42003,Clear,0,Pennsylvania,2011,9,42.003
9,42003,Clear,0,Pennsylvania,2011,10,42.003


In [70]:
test10 = final(data10,2010)
test10 = clean_state(test10)
test10 = clean_weather(test10)

In [71]:
test10.head(15)

,unit,weather,crashes,state,year,week,conv
0,42003,Clear,0,Pennsylvania,2010,1,42.003
1,42003,Rain,2,Pennsylvania,2010,2,42.003
2,42003,Clear,1,Pennsylvania,2010,3,42.003
3,42003,Clear,0,Pennsylvania,2010,4,42.003
4,42003,Clear,0,Pennsylvania,2010,5,42.003
5,42003,Clear,0,Pennsylvania,2010,6,42.003
6,42003,Clear,0,Pennsylvania,2010,7,42.003
7,42003,Snow,2,Pennsylvania,2010,8,42.003
8,42003,Clear,2,Pennsylvania,2010,9,42.003
9,42003,Clear,0,Pennsylvania,2010,10,42.003


In [72]:
finals = [test15,test14,test13,test12,test11,test10]
final = pd.concat(finals)

In [74]:
final.tail()

,unit,weather,crashes,state,year,week,conv
166415,31007,Clear,0,Nebraska,2010,49,31.007
166416,31007,Clear,0,Nebraska,2010,50,31.007
166417,31007,Clear,0,Nebraska,2010,51,31.007
166418,31007,Clear,0,Nebraska,2010,52,31.007
166419,31007,Clear,0,Nebraska,2010,53,31.007


In [75]:
final.to_csv('/Users/tristanmoser/files/tristan/488/Project/NHSA/weather/2weeksleft.csv',index=False)

In [76]:
len(final)

998520

In [77]:
final1 = final.drop(final[(final['state'] == 0)].index)

In [78]:
len(final1)

998202

In [79]:
tax = pd.read_csv('/Users/tristanmoser/files/tristan/488/project/NHSA/weather/state_exp.csv')
pop = pd.read_csv('/Users/tristanmoser/files/tristan/488/project/NHSA/weather/Populations.csv')

In [80]:
tax.head()

,state,Year,Tax_On_Fuel,Tax_on_License,Tot_Hwy_Exp
0,United States,2000,"$29,967,910","$32,598,497","$61,941,936"
1,United States,2001,"$31,025,871","$32,866,254","$66,435,728"
2,United States,2002,"$31,968,036","$35,391,102","$71,055,977"
3,United States,2003,"$32,269,077","$35,863,173","$72,454,881"
4,United States,2004,"$33,762,419","$39,707,951","$70,922,753"


In [81]:
tax15 = tax[tax['Year'] == '2015']
tax14 = tax[tax['Year'] == '2014']
tax13 = tax[tax['Year'] == '2013']
tax12 = tax[tax['Year'] == '2012']
tax11 = tax[tax['Year'] == '2011']
tax10 = tax[tax['Year'] == '2010']

In [82]:
total15 = pd.merge(test15,tax15,on='state')
total14 = pd.merge(test14,tax14,on='state')
total13 = pd.merge(test13,tax13,on='state')
total12 = pd.merge(test12,tax12,on='state')
total11 = pd.merge(test11,tax11,on='state')
total10 = pd.merge(test10,tax10,on='state')

In [83]:
total15.head()

,unit,weather,crashes,state,year,week,conv,Year,Tax_On_Fuel,Tax_on_License,Tot_Hwy_Exp
0,42003,Other,1,Pennsylvania,2015,1,42.003,2015,"$2,731,605","$2,222,062","$6,956,419"
1,42003,Other,0,Pennsylvania,2015,2,42.003,2015,"$2,731,605","$2,222,062","$6,956,419"
2,42003,Other,0,Pennsylvania,2015,3,42.003,2015,"$2,731,605","$2,222,062","$6,956,419"
3,42003,Clear,1,Pennsylvania,2015,4,42.003,2015,"$2,731,605","$2,222,062","$6,956,419"
4,42003,Clear,1,Pennsylvania,2015,5,42.003,2015,"$2,731,605","$2,222,062","$6,956,419"


In [84]:
len(test15)

166420

In [85]:
len(total15)

166314

In [91]:
totals15 = pd.merge(total15,pop[pop['Year']==2015],on='state')
totals14 = pd.merge(total14,pop[pop['Year']==2014],on='state')
totals13 = pd.merge(total13,pop[pop['Year']==2013],on='state')
totals12 = pd.merge(total12,pop[pop['Year']==2012],on='state')
totals11 = pd.merge(total11,pop[pop['Year']==2011],on='state')
totals10 = pd.merge(total10,pop[pop['Year']==2010],on='state')

In [96]:
totals15.tail(5)

,unit,weather,crashes,state,year,week,conv,Year_x,Tax_On_Fuel,Tax_on_License,Tot_Hwy_Exp,Year_y,Population
166309,50017,Clear,0,Vermont,2015,49,50.017,2015,"$85,619","$111,527","$501,722",2015,626088
166310,50017,Clear,0,Vermont,2015,50,50.017,2015,"$85,619","$111,527","$501,722",2015,626088
166311,50017,Clear,0,Vermont,2015,51,50.017,2015,"$85,619","$111,527","$501,722",2015,626088
166312,50017,Clear,0,Vermont,2015,52,50.017,2015,"$85,619","$111,527","$501,722",2015,626088
166313,50017,Clear,0,Vermont,2015,53,50.017,2015,"$85,619","$111,527","$501,722",2015,626088


In [109]:
def dst(dat,week):
    dat['DST'] = 0
    for ii in range(len(dat)):
        if dat['week'][ii] == week:
            dat['DST'][ii] = 1
    dat['prox'] = 0
    for ii in range(len(dat)):
        dat['prox'][ii] = dat['week'][ii] - week
    return dat

In [110]:
tota15 = dst(totals15,11.0)
tota14 = dst(totals14,11.0)
tota13 = dst(totals13,11.0)
tota12 = dst(totals12,11.0)
tota11 = dst(totals11,12.0)
tota10 = dst(totals10,12.0)

In [112]:
tota10.head(15)

,unit,weather,crashes,state,year,week,conv,Year_x,Tax_On_Fuel,Tax_on_License,Tot_Hwy_Exp,Year_y,Population,DST,prox
0,42003,Clear,0,Pennsylvania,2010,1,42.003,2010,"$2,020,099","$2,546,905","$6,738,126",2010,12712343,0,-11
1,42003,Rain,2,Pennsylvania,2010,2,42.003,2010,"$2,020,099","$2,546,905","$6,738,126",2010,12712343,0,-10
2,42003,Clear,1,Pennsylvania,2010,3,42.003,2010,"$2,020,099","$2,546,905","$6,738,126",2010,12712343,0,-9
3,42003,Clear,0,Pennsylvania,2010,4,42.003,2010,"$2,020,099","$2,546,905","$6,738,126",2010,12712343,0,-8
4,42003,Clear,0,Pennsylvania,2010,5,42.003,2010,"$2,020,099","$2,546,905","$6,738,126",2010,12712343,0,-7
5,42003,Clear,0,Pennsylvania,2010,6,42.003,2010,"$2,020,099","$2,546,905","$6,738,126",2010,12712343,0,-6
6,42003,Clear,0,Pennsylvania,2010,7,42.003,2010,"$2,020,099","$2,546,905","$6,738,126",2010,12712343,0,-5
7,42003,Snow,2,Pennsylvania,2010,8,42.003,2010,"$2,020,099","$2,546,905","$6,738,126",2010,12712343,0,-4
8,42003,Clear,2,Pennsylvania,2010,9,42.003,2010,"$2,020,099","$2,546,905","$6,738,126",2010,12712343,0,-3
9,42003,Clear,0,Pennsylvania,2010,10,42.003,2010,"$2,020,099","$2,546,905","$6,738,126",2010,12712343,0,-2


In [113]:
finals = [tota15,tota14,tota13,tota12,tota11,tota10]

In [114]:
totals = pd.concat(finals)

In [115]:
totals = totals.drop(['Year_x','Year_y'],axis=1)

In [116]:
totals = totals.drop(totals[(totals['state'] == 'Arizona')].index)
totals = totals.drop(totals[(totals['state'] == 'District of Columbia')].index)
totals = totals.drop(totals[(totals['state'] == 'Hawaii')].index)

totals = totals.drop(totals[(totals['week'] == 0)].index)

In [118]:
totals.tail()

,unit,weather,crashes,state,year,week,conv,Tax_On_Fuel,Tax_on_License,Tot_Hwy_Exp,Population,DST,prox
166309,50017,Clear,0,Vermont,2010,49,50.017,"$99,278","$106,558","$337,990",625982,0,37
166310,50017,Clear,0,Vermont,2010,50,50.017,"$99,278","$106,558","$337,990",625982,0,38
166311,50017,Clear,0,Vermont,2010,51,50.017,"$99,278","$106,558","$337,990",625982,0,39
166312,50017,Clear,0,Vermont,2010,52,50.017,"$99,278","$106,558","$337,990",625982,0,40
166313,50017,Clear,0,Vermont,2010,53,50.017,"$99,278","$106,558","$337,990",625982,0,41


In [120]:
totals.to_csv("/Users/tristanmoser/files/tristan/488/project/NHSA/weather/Totals2wks.csv",index=False)

In [121]:
len(totals)

991842

In [122]:
len(totals['unit'])

991842

In [124]:
totals[(totals['state'] == 'South Dakota') & (totals['year'] == 2011)]

,unit,weather,crashes,state,year,week,conv,Tax_On_Fuel,Tax_on_License,Tot_Hwy_Exp,Population,DST,prox
66515,46103,Rain,0,South Dakota,2011,1,46.103,"$127,437","$200,116","$562,115",824398,0,-11
66516,46103,Rain,0,South Dakota,2011,2,46.103,"$127,437","$200,116","$562,115",824398,0,-10
66517,46103,Rain,0,South Dakota,2011,3,46.103,"$127,437","$200,116","$562,115",824398,0,-9
66518,46103,Rain,0,South Dakota,2011,4,46.103,"$127,437","$200,116","$562,115",824398,0,-8
66519,46103,Rain,0,South Dakota,2011,5,46.103,"$127,437","$200,116","$562,115",824398,0,-7
66520,46103,Rain,0,South Dakota,2011,6,46.103,"$127,437","$200,116","$562,115",824398,0,-6
66521,46103,Rain,0,South Dakota,2011,7,46.103,"$127,437","$200,116","$562,115",824398,0,-5
66522,46103,Cloudy,1,South Dakota,2011,8,46.103,"$127,437","$200,116","$562,115",824398,0,-4
66523,46103,Cloudy,0,South Dakota,2011,9,46.103,"$127,437","$200,116","$562,115",824398,0,-3
66524,46103,Cloudy,0,South Dakota,2011,10,46.103,"$127,437","$200,116","$562,115",824398,0,-2
